## INTRODUCTION/BUSINESS PROBLEM

Two of most famous cities in the world are Toronto and New York. Both cities are very diverse, multicultural and are the financial capitals of their respective countries. I want to explore the similarity and dissimilarity of these cities about their touristic places. Tourism is an important activity for economy of countries and people usually like to visit developed countries. It is easy to explore and analyze cities with location data. Tourists like to visit different places and analysis of two cities will help them to make their choices easier and more efficient.

## DATA DESCRIPTION

I will use Foursquare API and get location data to explore the neighborhoods of two cities. This data gives information about different places such as coffee shops, restaurants, hotels, museums and many more around each neighborhood. I selected Scarborough from Toronto and Brooklyn from New York to analyze their neighborhoods. I will use k-means clustering technique to segment the neighborhoods regarding their places. This will help to understand their similarity and dissimilarity.

## PREPROCESSING

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
!pip install folium
!pip install BeautifulSoup4
!pip install geocoder

     |████████████████████████████████| 92kB 22.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 23.0MB/s ta 0:00:01


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

# To see full dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

from bs4 import BeautifulSoup # library to for web scraping

import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Visualization
import matplotlib.pyplot
import seaborn as sns

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Load and explore Toronto data

In [3]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

soup = BeautifulSoup(source, 'html5lib')

table = soup.find('table')

### Tranform the data into a pandas dataframe

In [4]:
column_names = ['PostalCode','Borough','Neighborhood']
df_toronto = pd.DataFrame(columns = column_names)

In [5]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df_toronto.loc[len(df_toronto)] = row_data

In [6]:
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
# Let's drop cells with a borough that is Not assigned
df_toronto=df_toronto[df_toronto['Borough']!='Not assigned']
df_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
df_toronto.shape

(103, 3)

### Get the latitude and longitude values of Toronto

In [9]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Let's merge both dataframes.
geo_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_merged = pd.merge(geo_df, df_toronto, on='PostalCode')
geo_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,Malvern / Rouge
1,M1C,43.784535,-79.160497,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,43.763573,-79.188711,Scarborough,Guildwood / Morningside / West Hill
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [11]:
# Sorting the columns
geo_data=geo_merged[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
# We select Scarborough from Toronto
scarborough_data=geo_data[geo_data['Borough'].str.contains("Scarborough")]

# Drop PostalCode column
scarborough_data=scarborough_data.drop(['PostalCode'], axis=1)

scarborough_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
scarborough_data.shape

(17, 4)

### Load and Explore New York Data

In [14]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [15]:
# Let's load the data.
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [16]:
# Let's take a quick look at the data.
newyork_data

{'type': 'FeatureCollection',
 'totalFeatures': 306,
 'features': [{'type': 'Feature',
   'id': 'nyu_2451_34572.1',
   'geometry': {'type': 'Point',
    'coordinates': [-73.84720052054902, 40.89470517661]},
   'geometry_name': 'geom',
   'properties': {'name': 'Wakefield',
    'stacked': 1,
    'annoline1': 'Wakefield',
    'annoline2': None,
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.84720052054902,
     40.89470517661,
     -73.84720052054902,
     40.89470517661]}},
  {'type': 'Feature',
   'id': 'nyu_2451_34572.2',
   'geometry': {'type': 'Point',
    'coordinates': [-73.82993910812398, 40.87429419303012]},
   'geometry_name': 'geom',
   'properties': {'name': 'Co-op City',
    'stacked': 2,
    'annoline1': 'Co-op',
    'annoline2': 'City',
    'annoline3': None,
    'annoangle': 0.0,
    'borough': 'Bronx',
    'bbox': [-73.82993910812398,
     40.87429419303012,
     -73.82993910812398,
     40.87429419303012]}},
  {'type': 'Feature',
 

In [17]:
# Let's define a new variable that includes all the relevant data which is in features key.
neighborhoods_data = newyork_data['features']

In [18]:
# Let's take a look at the first item in this list.
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a *pandas* dataframe

In [19]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [20]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [21]:
# Let's loop through the data and fill the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [22]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [23]:
# And make sure that the dataset has all 5 boroughs and 306 neighborhoods.
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [24]:
# We select Brooklyn from New York
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [25]:
brooklyn_data.shape

(70, 4)

## Foursquare API

In [26]:
# Define Foursquare Credentials and Version
CLIENT_ID = '5RC1XK0WIJNWJGVKTB2EYS3WB4RX3EKZ4V2AA1BN4KDKT3HE' # your Foursquare ID
CLIENT_SECRET = 'EAPDQJHIFVN0QM20DEMDILDJNQI2JFO5K52XW1CUYO4U3OGQ' # your Foursquare Secret
VERSION = '20180604'
limit = 20

print('Your credentails:')
print('CLIENT_ID:'+ CLIENT_ID)
print('CLIENT_SECRET:'+ CLIENT_SECRET)

Your credentails:
CLIENT_ID:5RC1XK0WIJNWJGVKTB2EYS3WB4RX3EKZ4V2AA1BN4KDKT3HE
CLIENT_SECRET:EAPDQJHIFVN0QM20DEMDILDJNQI2JFO5K52XW1CUYO4U3OGQ


In [27]:
# get the geographical coordinates of Downtown Toronto
address = 'Scarborough, ON, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_scarborough = location.latitude
longitude_scarborough = location.longitude
print("Scarborough","latitude",latitude_scarborough, "& " "longitude" ,longitude_scarborough)

Scarborough latitude 43.773077 & longitude -79.257774


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [28]:
# Let's get the geographical coordinates of Manhattan.
address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


## VISUALIZATION OF NEIGHBORHOODS OF SCARBOROUGH AND BROOKLYN

In [29]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude_scarborough, longitude_scarborough], zoom_start=11)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

In [30]:
# create map of Brooklyn using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

## ANALYSIS 

We analyze neighborhoods of both boroughs through one hot encoding (giving ‘1’ if a venue category is there, and ‘0’ in case of venue category is not there). On the basis of one hot encoding, we calculate the frequency of each category and pick top ten venues on that basis for each neighborhood. It means the top venues are showing the foot traffic or the more visited places.

### Exploring Neighborhoods in Scarborough

In [31]:
# Let's create a function to repeat the same process to all the neighborhoods in Toronto.
def getNearbyVenues(names, latitudes,longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [32]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called scarborough_venues.
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern / Rouge
Rouge Hill / Port Union / Highland Creek
Guildwood / Morningside / West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park / Ionview / East Birchmount Park
Golden Mile / Clairlea / Oakridge
Cliffside / Cliffcrest / Scarborough Village West
Birch Cliff / Cliffside West
Dorset Park / Wexford Heights / Scarborough Town Centre
Wexford / Maryvale
Agincourt
Clarks Corners / Tam O'Shanter / Sullivan
Milliken / Agincourt North / Steeles East / L'Amoreaux East
Steeles West / L'Amoreaux West
Upper Rouge


In [33]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Malvern / Rouge,43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
3,Guildwood / Morningside / West Hill,43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,Guildwood / Morningside / West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [34]:
# Let's check how many venues were returned for each neighborhood
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Birch Cliff / Cliffside West,4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
Clarks Corners / Tam O'Shanter / Sullivan,16,16,16,16,16,16
Cliffside / Cliffcrest / Scarborough Village West,2,2,2,2,2,2
Dorset Park / Wexford Heights / Scarborough Town Centre,6,6,6,6,6,6
Golden Mile / Clairlea / Oakridge,10,10,10,10,10,10
Guildwood / Morningside / West Hill,7,7,7,7,7,7
Kennedy Park / Ionview / East Birchmount Park,8,8,8,8,8,8


In [35]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 58 uniques categories.


### Analyzing Each Neighborhood

In [36]:
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Women's Store
0,Malvern / Rouge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Rouge Hill / Port Union / Highland Creek,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Rouge Hill / Port Union / Highland Creek,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Guildwood / Morningside / West Hill,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Guildwood / Morningside / West Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [37]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Camera Store,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Nail Salon,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Supermarket,Thai Restaurant,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Women's Store
0,Agincourt,0.0,0.000,0.000,0.0000,0.0,0.25,0.0,0.0,0.00,0.0,0.000,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.00,0.0,0.000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.25,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000,0.000,0.0,0.0000,0.0,0.0000,0.25,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0
1,Birch Cliff / Cliffside West,0.0,0.000,0.000,0.0000,0.0,0.00,0.0,0.0,0.25,0.0,0.000,0.0000,0.0000,0.25,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.25,0.0,0.000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000,0.000,0.0,0.0000,0.0,0.0000,0.25,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0
2,Cedarbrae,0.0,0.125,0.125,0.1250,0.0,0.00,0.0,0.0,0.00,0.0,0.125,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.1250,0.0,0.1250,0.00,0.0,0.125,0.0,0.0,0.0,0.0000,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0,0.0,0.0,0.1250,0.0,0.0,0.0,0.0
3,Clarks Corners / Tam O'Shanter / Sullivan,0.0,0.000,0.000,0.0625,0.0,0.00,0.0,0.0,0.00,0.0,0.000,0.0625,0.0625,0.00,0.0,0.0,0.0,0.0,0.0,0.0625,0.0625,0.0,0.0625,0.00,0.0,0.000,0.0,0.0,0.0,0.0625,0.0625,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,0.125,0.125,0.0,0.0625,0.0,0.0625,0.00,0.0,0.0,0.0,0.0625,0.0,0.0,0.0,0.0
4,Cliffside / Cliffcrest / Scarborough Village West,0.5,0.000,0.000,0.0000,0.0,0.00,0.0,0.0,0.00,0.0,0.000,0.0000,0.0000,0.00,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.00,0.0,0.000,0.0,0.0,0.0,0.0000,0.0000,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.5,0.0,0.0000,0.0,0.0,0.000,0.000,0.0,0.0000,0.0,0.0000,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0


In [38]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Birch Cliff / Cliffside West----
                   venue  freq
0  General Entertainment  0.25
1                   Café  0.25
2           Skating Rink  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0               Bakery  0.12
1                 Bank  0.12
2  Fried Chicken Joint  0.12
3   Athletics & Sports  0.12
4      Thai Restaurant  0.12


----Clarks Corners / Tam O'Shanter / Sullivan----
                  venue  freq
0              Pharmacy  0.12
1           Pizza Place  0.12
2          Intersection  0.06
3    Chinese Restaurant  0.06
4  Fast Food Restaurant  0.06


----Cliffside / Cliffcrest / Scarborough Village West----
                       venue  freq
0        American Restaura

### Let's put that into a pandas dataframe

In [39]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Skating Rink,Women's Store,Construction & Landscaping,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store
1,Birch Cliff / Cliffside West,General Entertainment,Skating Rink,Café,College Stadium,Women's Store,Construction & Landscaping,Hakka Restaurant,Grocery Store,Gas Station,Furniture / Home Store
2,Cedarbrae,Caribbean Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Women's Store,Discount Store
3,Clarks Corners / Tam O'Shanter / Sullivan,Pizza Place,Pharmacy,Fast Food Restaurant,Gas Station,Coffee Shop,Thai Restaurant,Bank,Fried Chicken Joint,Intersection,Italian Restaurant
4,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,Construction & Landscaping,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


### Clustering Neighborhoods of Scarborough

In [41]:
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe 
kmeans.labels_

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 2, 1, 0, 3, 1, 1, 1], dtype=int32)

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,Fast Food Restaurant,Women's Store,Construction & Landscaping,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
1,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Women's Store,Hobby Shop,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
2,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711,1.0,Medical Center,Intersection,Bank,Electronics Store,Breakfast Spot,Mexican Restaurant,Rental Car Location,Women's Store,Discount Store,Fast Food Restaurant
3,Scarborough,Woburn,43.770992,-79.216917,1.0,Coffee Shop,Indian Restaurant,Korean Restaurant,Construction & Landscaping,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
4,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Caribbean Restaurant,Athletics & Sports,Bakery,Thai Restaurant,Bank,Hakka Restaurant,Gas Station,Fried Chicken Joint,Women's Store,Discount Store


In [43]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], np.int_(scarborough_merged['Cluster Labels'][0:16])):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters 

Now, we can examine each cluster and determine venue categories that distinguish each cluster. Based on the defining categories, we can assign a name to each cluster.

### Cluster 1 (Health & Beauty Center, Park, Playground)

In [44]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 0, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,Health & Beauty Service,Playground,Coffee Shop,Grocery Store,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant
14,Milliken / Agincourt North / Steeles East / L'...,Park,Playground,Chinese Restaurant,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


### Cluster 2 (Bank, Coffee Shop, Breakfast Spot)

In [45]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 1, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Guildwood / Morningside / West Hill,Bank,Intersection,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Convenience Store,General Entertainment
3,Woburn,Coffee Shop,Korean Restaurant,Convenience Store,College Stadium,Grocery Store,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint
4,Cedarbrae,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Bakery,Bank,Hakka Restaurant,Caribbean Restaurant,Gas Station,Vietnamese Restaurant,Electronics Store
6,Kennedy Park / Ionview / East Birchmount Park,Coffee Shop,Chinese Restaurant,Discount Store,Department Store,Train Station,Bank,Bar,Gym,Grocery Store,Golf Course
7,Golden Mile / Clairlea / Oakridge,Bakery,Bus Line,Metro Station,Soccer Field,Ice Cream Shop,Park,Intersection,Bus Station,Fast Food Restaurant,Department Store
9,Birch Cliff / Cliffside West,College Stadium,General Entertainment,Skating Rink,Café,Vietnamese Restaurant,Grocery Store,Golf Course,Gas Station,Furniture / Home Store,Fried Chicken Joint
10,Dorset Park / Wexford Heights / Scarborough To...,Indian Restaurant,Vietnamese Restaurant,Brewery,Furniture / Home Store,Pet Store,Chinese Restaurant,Bar,Department Store,Golf Course,Athletics & Sports
11,Wexford / Maryvale,Breakfast Spot,Bakery,Sandwich Place,Shopping Mall,Middle Eastern Restaurant,Auto Garage,General Entertainment,Gas Station,Golf Course,College Stadium
12,Agincourt,Breakfast Spot,Latin American Restaurant,Lounge,Chinese Restaurant,Vietnamese Restaurant,Convenience Store,Golf Course,General Entertainment,Gas Station,Furniture / Home Store
13,Clarks Corners / Tam O'Shanter / Sullivan,Pharmacy,Pizza Place,Noodle House,Fast Food Restaurant,Coffee Shop,Intersection,Italian Restaurant,Bank,Fried Chicken Joint,Thai Restaurant


### Cluster 3 (Fast Food Restaurant, Vietnamese Restaurant, Hakka Restaurant)

In [46]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 2, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Malvern / Rouge,Fast Food Restaurant,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Electronics Store


### Cluster 4 (American Restaurant, Motel, Stadium)

In [48]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 3, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Cliffside / Cliffcrest / Scarborough Village West,American Restaurant,Motel,College Stadium,Grocery Store,Golf Course,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


### Cluster 5 (Golf Course, Bar, Vietnamese Restaurant)

In [49]:
scarborough_merged.loc[scarborough_merged['Cluster Labels'] == 4, scarborough_merged.columns[[1] + list(range(5, scarborough_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Rouge Hill / Port Union / Highland Creek,Golf Course,Bar,Vietnamese Restaurant,College Stadium,Grocery Store,General Entertainment,Gas Station,Furniture / Home Store,Fried Chicken Joint,Fast Food Restaurant


### Exploring Neighborhoods in Brooklyn

In [31]:
# Let's create a function to repeat the same process to all the neighborhoods in Brooklyn
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [32]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called brooklyn_venues
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude'],
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [33]:
brooklyn_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place
3,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
4,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot


In [52]:
# Let's check how many venues were returned for each neighborhood
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,20,20,20,20,20,20
Bay Ridge,20,20,20,20,20,20
Bedford Stuyvesant,20,20,20,20,20,20
Bensonhurst,20,20,20,20,20,20
Bergen Beach,6,6,6,6,6,6
Boerum Hill,20,20,20,20,20,20
Borough Park,20,20,20,20,20,20
Brighton Beach,20,20,20,20,20,20
Broadway Junction,11,11,11,11,11,11


In [53]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 234 uniques categories.


### Analyzing the Neighborhoods

In [54]:
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Entertainment Service,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Insurance Office,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Non-Profit,Noodle House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Varenyky restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Bath Beach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.00,0.05,0.0,0.00,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.1,0.0,0.05,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.05,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.00,0.05,0.0,0.0,0.0,0.0,0.1,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,

In [56]:
brooklyn_grouped.shape

(70, 234)

In [57]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0  Italian Restaurant  0.10
1     Bubble Tea Shop  0.10
2      Ice Cream Shop  0.05
3    Sushi Restaurant  0.05
4          Hookah Bar  0.05


----Bay Ridge----
              venue  freq
0               Spa  0.10
1     Grocery Store  0.10
2  Greek Restaurant  0.10
3            Lounge  0.05
4         Juice Bar  0.05


----Bedford Stuyvesant----
         venue  freq
0         Café  0.10
1  Coffee Shop  0.10
2          Bar  0.10
3    BBQ Joint  0.05
4     Boutique  0.05


----Bensonhurst----
                venue  freq
0      Ice Cream Shop  0.10
1  Italian Restaurant  0.10
2    Sushi Restaurant  0.10
3  Chinese Restaurant  0.10
4             Butcher  0.05


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.33
1  Athletics & Sports  0.17
2      Baseball Field  0.17
3          Playground  0.17
4          Donut Shop  0.17


----Boerum Hill----
                    venue  freq
0                     Bar  0.10
1  Furniture / 

### Let's put that into a pandas dataframe

In [58]:
# First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Italian Restaurant,Bubble Tea Shop,Ice Cream Shop,Sushi Restaurant,German Restaurant,Park,Peruvian Restaurant,Dim Sum Restaurant,Dessert Shop,Pizza Place
1,Bay Ridge,Spa,Grocery Store,Greek Restaurant,Juice Bar,Breakfast Spot,Sports Bar,Bookstore,Caucasian Restaurant,Lounge,Taco Place
2,Bedford Stuyvesant,Coffee Shop,Café,Bar,Fried Chicken Joint,BBQ Joint,Park,Cocktail Bar,New American Restaurant,Boutique,Gift Shop
3,Bensonhurst,Ice Cream Shop,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Bakery,Cosmetics Shop,Donut Shop,Russian Restaurant,Noodle House,Shabu-Shabu Restaurant
4,Bergen Beach,Harbor / Marina,Donut Shop,Athletics & Sports,Baseball Field,Playground,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field


### Clustering Neighborhoods of Brooklyn

In [60]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 2, 0, 1, 2, 1, 0, 1, 2], dtype=int32)

In [61]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,2,Spa,Grocery Store,Greek Restaurant,Juice Bar,Breakfast Spot,Sports Bar,Bookstore,Caucasian Restaurant,Lounge,Taco Place
1,Brooklyn,Bensonhurst,40.611009,-73.995180,0,Ice Cream Shop,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Bakery,Cosmetics Shop,Donut Shop,Russian Restaurant,Noodle House,Shabu-Shabu Restaurant
2,Brooklyn,Sunset Park,40.645103,-74.010316,0,Pizza Place,Latin American Restaurant,Bakery,Bank,Mexican Restaurant,Ice Cream Shop,Café,Creperie,Breakfast Spot,Grocery Store
3,Brooklyn,Greenpoint,40.730201,-73.954241,2,Cocktail Bar,Café,Yoga Studio,Beer Store,Gymnastics Gym,Grocery Store,Furniture / Home Store,Mexican Restaurant,Deli / Bodega,Pizza Place
4,Brooklyn,Gravesend,40.595260,-73.973471,0,Lounge,Pizza Place,Bakery,Italian Restaurant,Pharmacy,Doctor's Office,Donut Shop,Music Store,Sporting Goods Shop,Furniture / Home Store


In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters of Brooklyn Neighborhoods

### Cluster 1 (Pizza Place, Restaurant, Pub)

In [64]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Bensonhurst,Ice Cream Shop,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Bakery,Cosmetics Shop,Donut Shop,Russian Restaurant,Noodle House,Shabu-Shabu Restaurant
2,Sunset Park,Pizza Place,Latin American Restaurant,Bakery,Bank,Mexican Restaurant,Ice Cream Shop,Café,Creperie,Breakfast Spot,Grocery Store
4,Gravesend,Lounge,Pizza Place,Bakery,Italian Restaurant,Pharmacy,Doctor's Office,Donut Shop,Music Store,Sporting Goods Shop,Furniture / Home Store
5,Brighton Beach,Pharmacy,Restaurant,Mediterranean Restaurant,Food & Drink Shop,Bookstore,Supermarket,Sushi Restaurant,Taco Place,Beach,Korean Restaurant
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Yoga Studio,Italian Restaurant,Diner,Outlet Store,Restaurant,Russian Restaurant,Miscellaneous Shop
9,Crown Heights,Pizza Place,Museum,Café,Pharmacy,Candy Store,Burger Joint,Coffee Shop,Bakery,Bagel Shop,Supermarket
11,Kensington,Thai Restaurant,Ice Cream Shop,Grocery Store,Pizza Place,Liquor Store,Mobile Phone Shop,Bakery,Bagel Shop,Outdoors & Recreation,Japanese Restaurant
14,Brownsville,Restaurant,Moving Target,Park,Pharmacy,Discount Store,Farmers Market,Spanish Restaurant,Fried Chicken Joint,Chinese Restaurant,Trail
22,Gowanus,Pizza Place,Art Gallery,Italian Restaurant,Coffee Shop,Music Venue,Bar,Brewery,Grocery Store,Pie Shop,Furniture / Home Store
31,Manhattan Beach,Café,Bus Stop,Ice Cream Shop,Beach,Sandwich Place,Food,Playground,Home Service,Hardware Store,Fish Market


### Cluster 2 (Bank, Gym, Caribbean Restaurant)

In [65]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Grocery Store,Mobile Phone Shop,Steakhouse,Organic Grocery,Coffee Shop,Bank,Convenience Store
8,Flatbush,Bank,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Pharmacy,Plaza,Deli / Bodega,Donut Shop,Chinese Restaurant
10,East Flatbush,Department Store,Park,Food & Drink Shop,Moving Target,Fast Food Restaurant,Supermarket,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Hardware Store
25,Cypress Hills,Latin American Restaurant,Fried Chicken Joint,Ice Cream Shop,Fast Food Restaurant,Pizza Place,Liquor Store,Donut Shop,Supermarket,Gas Station,Chinese Restaurant
26,East New York,Deli / Bodega,Event Service,Metro Station,Salon / Barbershop,Gym,Fast Food Restaurant,Fried Chicken Joint,Food Truck,Pharmacy,Plaza
27,Starrett City,Pizza Place,Bus Station,Donut Shop,Shopping Mall,Convenience Store,Caribbean Restaurant,Pharmacy,American Restaurant,Fish & Chips Shop,Factory
28,Canarsie,Food,Shipping Store,Grocery Store,Gym,Caribbean Restaurant,Asian Restaurant,Chinese Restaurant,Bus Line,Fast Food Restaurant,Field
29,Flatlands,Pharmacy,Caribbean Restaurant,Fast Food Restaurant,Fried Chicken Joint,Nightclub,Seafood Restaurant,Lounge,Bar,Chinese Restaurant,Electronics Store
32,Coney Island,Caribbean Restaurant,Baseball Stadium,Brewery,Theme Park Ride / Attraction,Pharmacy,Music Venue,Skating Rink,Monument / Landmark,Food Court,Beach
34,Borough Park,Bank,Fast Food Restaurant,Pharmacy,Pizza Place,Deli / Bodega,Hotel,Metro Station,Restaurant,Coffee Shop,Sandwich Place


### Cluster 3 (Bar, Theater, Coffee Shop)

In [67]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Spa,Grocery Store,Greek Restaurant,Juice Bar,Breakfast Spot,Sports Bar,Bookstore,Caucasian Restaurant,Lounge,Taco Place
3,Greenpoint,Cocktail Bar,Café,Yoga Studio,Beer Store,Gymnastics Gym,Grocery Store,Furniture / Home Store,Mexican Restaurant,Deli / Bodega,Pizza Place
12,Windsor Terrace,Diner,Grocery Store,Café,Middle Eastern Restaurant,Italian Restaurant,Sushi Restaurant,Bookstore,Chinese Restaurant,Bar,Thrift / Vintage Store
13,Prospect Heights,Bar,Ice Cream Shop,Cocktail Bar,Playground,Coffee Shop,New American Restaurant,Caribbean Restaurant,Cajun / Creole Restaurant,Café,Brewery
15,Williamsburg,Bar,Coffee Shop,Yoga Studio,Liquor Store,Korean Restaurant,Taco Place,Tapas Restaurant,Steakhouse,Lounge,Event Space
16,Bushwick,Bar,Mexican Restaurant,Thrift / Vintage Store,Coffee Shop,Used Bookstore,French Restaurant,Sandwich Place,Latin American Restaurant,Korean Restaurant,Bagel Shop
17,Bedford Stuyvesant,Coffee Shop,Café,Bar,Fried Chicken Joint,BBQ Joint,Park,Cocktail Bar,New American Restaurant,Boutique,Gift Shop
18,Brooklyn Heights,Yoga Studio,Pet Store,Gym / Fitness Center,Playground,Pizza Place,Mexican Restaurant,Japanese Restaurant,History Museum,Gym,Thai Restaurant
19,Cobble Hill,Italian Restaurant,Bar,Cocktail Bar,Deli / Bodega,Seafood Restaurant,Men's Store,Spa,Spanish Restaurant,French Restaurant,Bookstore
20,Carroll Gardens,Coffee Shop,Farmers Market,Arts & Crafts Store,Bar,Cocktail Bar,Bakery,Greek Restaurant,Restaurant,Spa,Italian Restaurant


### Cluster 4 (Pool, Wings Joint, Event Space)

In [69]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Wings Joint,Event Service,Food & Drink Shop,Food,Flower Shop,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant


### Cluster 5 (Fish Market, Moving Target, Harbor/Marina)

In [70]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
59,Paerdegat Basin,Food,Moving Target,Bus Line,Harbor / Marina,Asian Restaurant,Wings Joint,Event Space,Flower Shop,Fish Market,Fish & Chips Shop


## RESULTS

After clustering the data of the respective neighborhoods, it's clear that Brooklyn has more venues which can be explored and attract the tourists. The neighborhoods are much similar in features like clubs, coffee shops etc. As far as concern to dissimilarity, it differs in terms of some unique places like restaurants, sport places, historical places.

## OBSERVATIONS AND RECOMMENDATIONS

When we compare the tourist places, we observe that Brooklyn has more different venues. Different kind of stores, stadiums, coffee shops are available in Scarborough while venues like nightlife, restaurants, museums and theaters are present in Brooklyn.

As far as concern to recommendations, we recommend Brooklyn neighborhoods will be considered first to visit. The tourists will have the opportunity to explore different kind of venues. We recommend you to visit Scarborough just in case of you have enough time and want to explore a different city and borough.


## CONCLUSION 

Brooklyn is a more diverse borough in terms of locations compared to Scarborough. As we know that every place is unique in its own way, so that’s argument is present in both neighborhoods. The dissimilarity exists in terms of some different venues and facilities on a large extent.